# Import packages

In [1]:
import os
import warnings
from statistics import median

import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
from shapely import LineString, Point
from datetime import timedelta

# Notebook settings

In [2]:
GTFS_FOLDER = 'GTFS_20190914'
LINES = ['RED', 'BLUE', 'GREEN', 'YELLOW', 'ORANGE', 'SILVER']

In [3]:
# Settings processing
if not os.path.exists(f"Intermediates\\{GTFS_FOLDER}"):
    os.makedirs(f"Intermediates\\{GTFS_FOLDER}")

# Load in data

In [4]:
# Service dates
calendar_dates = pd.read_csv(f"Data\\{GTFS_FOLDER}\\calendar_dates.txt", sep=',')
calendar_dates['date'] = pd.to_datetime(calendar_dates['date'], format='%Y%m%d')

dates = pd.DataFrame(columns=['service_id', 'dates'])
for service_id in calendar_dates['service_id'].unique():
    dates.loc[len(dates)] = [service_id, calendar_dates[calendar_dates['service_id'] == service_id]['date'].to_list()]
print(len(dates))
dates.head(3)

28


,service_id,dates
0,1,[2019-10-14 00:00:00]
1,2,[2019-09-21 00:00:00]
2,3,"[2019-09-28 00:00:00, 2019-10-05 00:00:00, 201..."


In [5]:
# Trips
trips = pd.read_csv(f"Data\\{GTFS_FOLDER}\\trips.txt", sep=',')
trips = trips[trips['route_id'].isin(LINES)].copy()

print(len(trips))
trips.head(3)

17067


C:\Users\lange\AppData\Local\Temp\ipykernel_25504\2785772606.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(f"Data\\{GTFS_FOLDER}\\trips.txt", sep=',')


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,scheduled_trip_id
0,RED,1,3037507_18184,GLENMONT,0,NaN,1,3037507.0
1,RED,1,3037508_18184,GLENMONT,0,NaN,1,3037508.0
2,RED,1,3037509_18184,GLENMONT,0,NaN,1,3037509.0


In [6]:
# Stop times
stop_times_data = pd.read_csv(f"Data\\{GTFS_FOLDER}\\stop_times.txt", sep=',')
stop_times = stop_times_data[stop_times_data['trip_id'].isin(trips['trip_id'])].copy()

print(len(stop_times))
stop_times.head(3)

C:\Users\lange\AppData\Local\Temp\ipykernel_25504\3131791201.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times_data = pd.read_csv(f"Data\\{GTFS_FOLDER}\\stop_times.txt", sep=',')


409090


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
0,3037507_18184,05:00:00,05:00:00,9360,1,0,0,0.0000
1,3037507_18184,05:04:00,05:04:00,7993,2,0,0,2.5847
2,3037507_18184,05:07:00,05:07:00,3969,3,0,0,4.7091


In [10]:
# stations
station_data = pd.read_csv(f"Data\\{GTFS_FOLDER}\\stops.txt", sep=',')
# Filter out stops without stop times
stations = station_data[station_data['stop_id'].isin(stop_times['stop_id'])].copy()
# Convert stations coords to geom objects
stations['geometry'] = [Point(station['stop_lon'], station['stop_lat']) for _, station in stations.iterrows()]
stations = gpd.GeoDataFrame(stations, geometry='geometry')

stations = stations[['stop_id', 'stop_name', 'geometry']].copy()
print(len(stations))
stations.head(3)

91


,stop_id,stop_name,geometry
167,308,SHAW METRO STATION,POINT (-77.02193 38.91455)
476,999,CHEVERLY METRO STATION,POINT (-76.9151 38.91655)
648,1305,CAPITOL HEIGHTS METRO STATION,POINT (-76.91331 38.88957)


In [8]:
# Track segment shapes
shapes_data = pd.read_csv(f"Data\\{GTFS_FOLDER}\\shapes.txt", sep=',')
shapes_data = shapes_data[shapes_data['shape_id'].isin(trips['shape_id'])]

route_shapes = gpd.GeoDataFrame(columns=['shape_id', 'length', 'geometry'], geometry='geometry')
points_per_shapeid = {shape_id[0]: shape_data for shape_id, shape_data in shapes_data .groupby(['shape_id'])}
for shape_id in shapes_data['shape_id'].unique():
    points = points_per_shapeid[shape_id]
    line = LineString([(point['shape_pt_lon'], point['shape_pt_lat']) for _, point in points.iterrows()])
    route_shapes.loc[len(route_shapes)] = [shape_id, points['shape_dist_traveled'].values[-1], line]

print(len(route_shapes))
route_shapes.head(3)

347


,shape_id,length,geometry
0,1,31.0492,"LINESTRING (-77.16476 39.11999, -77.14669 39.0..."
1,2,10.4316,"LINESTRING (-76.99568 38.92078, -76.99447 38.9..."
2,3,31.0492,"LINESTRING (-77.16476 39.11999, -77.14669 39.0..."


# Prepare stations

In [ ]:
# Find all connections between stations
stations['from_stops'] = [set() for _ in range(len(stations))]
stations['to_stops'] = [set() for _ in range(len(stations))]
stations['connections'] = [set() for _ in range(len(stations))]

for service_id in trips['service_id'].unique():
    routes = trips[trips['service_id'] == service_id]['shape_id'].unique()

    for route in routes:
        single_trip_id = trips[(trips['service_id'] == service_id) & (trips['shape_id'] == route)]['trip_id'].values[0]
        single_trip = stop_times[stop_times['trip_id'] == single_trip_id]

        current_stops = single_trip['stop_id'].values
        previous_stops = [None, *current_stops[:-1]]
        next_stops = [*current_stops[1:], None]

        for current_stop, prev_stop, next_stop in zip(current_stops, previous_stops, next_stops):
            
            current_stop_index = stations[stations['stop_id'] == current_stop].index[0]
            stations.at[current_stop_index, 'from_stops'].add(prev_stop)
            stations.at[current_stop_index, 'to_stops'].add(next_stop)
            stations.at[current_stop_index, 'connections'].add((prev_stop, next_stop))

# Remove all connections with NaN values (imaginary connections from the end of LINES)
for index, station in stations.iterrows():
    stations.at[index, 'from_stops'] = [int(station_i) for station_i in station['from_stops'] if pd.notna(station_i)]
    stations.at[index, 'to_stops'] = [int(station_i) for station_i in station['to_stops'] if pd.notna(station_i)]
    stations.at[index, 'connections'] = [[int(station_i) if pd.notna(station_i) else 'None' for station_i in connection_i] \
                                      for connection_i in station['connections']]

# Save file for later use
stations = stations.set_crs('EPSG:4326')
stations.to_file(f"Intermediates\\{GTFS_FOLDER}\\gtfs_stations.geojson", driver="GeoJSON")
stations.head(3)

c:\Users\lange\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


,stop_id,stop_name,geometry,from_stops,to_stops,connections
167,308,SHAW METRO STATION,POINT (-77.02193 38.91455),"[10344, 1418]","[10344, 1418]","[[1418, 10344], [10344, 1418]]"
476,999,CHEVERLY METRO STATION,POINT (-76.9151 38.91655),"[2124, 5030]","[2124, 5030]","[[5030, 2124], [2124, 5030]]"
648,1305,CAPITOL HEIGHTS METRO STATION,POINT (-76.91331 38.88957),"[13107, 4613]","[13107, 4613]","[[4613, 13107], [13107, 4613]]"


In [ ]:
# for index, station in stations.iterrows():
#     stations.at[index, 'from_stops'] = [int(station) for station in station['from_stops'] if pd.notna(station)]
#     stations.at[index, 'to_stops'] = [int(station) for station in station['to_stops'] if pd.notna(station)]
#     stations.at[index, 'connections'] = [[int(station) for station in connection] for connection in station['connections'] if \
#                                          pd.notna(connection[0]) and pd.notna(connection[1])]

# Prepare stops

In [10]:
stop_times.head(3)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
0,3037507_18184,05:00:00,05:00:00,9360,1,0,0,0.0000
1,3037507_18184,05:04:00,05:04:00,7993,2,0,0,2.5847
2,3037507_18184,05:07:00,05:07:00,3969,3,0,0,4.7091


In [11]:
# Convert dictionary of all arrival times and their corresponding timedelta representation
timedeltas = {}
for stop_time in stop_times['arrival_time'].unique():
    arrival_in_seconds = int(stop_time[:2]) * 3600 + int(stop_time[3:5]) * 60 + int(stop_time[6:])
    time = timedelta(0, arrival_in_seconds)
    timedeltas[stop_time] = time

In [12]:
# Create trip stop itinerary for each trip
trip_dfs = []
trip_split_stops = {trip_id[0]: trip_data for trip_id, trip_data in stop_times.groupby(['trip_id'])}
for i, (_, trip) in enumerate(trips.iterrows()):
    if i % 1000 == 0:
        print(f"{i:_} out of {len(trips):_} trips processed")

    trip_stops = trip_split_stops[trip['trip_id']]
    trip_stops_ids = [f"{trip_id}_{stop_id}" for trip_id, stop_id in \
                      zip(trip_stops['trip_id'].values, trip_stops['stop_id'].values)]

    trip_df = pd.DataFrame({'line_name': [trip['route_id']] * len(trip_stops),
                            'service_id': [trip['service_id']] * len(trip_stops),
                            'trip_id': [trip['trip_id']] * len(trip_stops),
                            'shape_id': [trip['shape_id']] * len(trip_stops),
                            'stop': trip_stops['stop_id'],
                            'arrival_time': [timedeltas[time] for time in trip_stops['arrival_time'].values],
                            'previous_stop': [None, *trip_stops['stop_id'].values[:-1]],
                            'next_stop': [*trip_stops['stop_id'].values[1:], None]})

    trip_dfs.append(trip_df)
print(f"{len(trips):_} out of {len(trips):_}")

trip_itineraries = pd.concat(trip_dfs)
trip_itineraries['previous_stop'] = trip_itineraries['previous_stop'].astype('Int64')
trip_itineraries['next_stop'] = trip_itineraries['next_stop'].astype('Int64')

trip_itineraries.head(3)

0 out of 17_067 trips processed
1_000 out of 17_067 trips processed
2_000 out of 17_067 trips processed
3_000 out of 17_067 trips processed
4_000 out of 17_067 trips processed
5_000 out of 17_067 trips processed
6_000 out of 17_067 trips processed
7_000 out of 17_067 trips processed
8_000 out of 17_067 trips processed
9_000 out of 17_067 trips processed
10_000 out of 17_067 trips processed
11_000 out of 17_067 trips processed
12_000 out of 17_067 trips processed
13_000 out of 17_067 trips processed
14_000 out of 17_067 trips processed
15_000 out of 17_067 trips processed
16_000 out of 17_067 trips processed
17_000 out of 17_067 trips processed
17_067 out of 17_067


,line_name,service_id,trip_id,shape_id,stop,arrival_time,previous_stop,next_stop
0,RED,1,3037507_18184,1,9360,0 days 05:00:00,<NA>,7993
1,RED,1,3037507_18184,1,7993,0 days 05:04:00,9360,3969
2,RED,1,3037507_18184,1,3969,0 days 05:07:00,7993,22094


In [13]:
# Apply each trip itinerary to all dates for which the trip was run
stop_datetimes_dfs = []
itineraries_per_service = {service_id[0]: itinerary_data for service_id, itinerary_data in trip_itineraries.groupby(['service_id'])}

for service_id, service_dates in zip(dates['service_id'].values, dates['dates'].values):
    if service_id not in itineraries_per_service.keys():
        # TODO: Service 20 is missing?
        continue

    service_itineraries = itineraries_per_service[service_id]
    for date in service_dates:
        date_x_df = service_itineraries.copy()
        date_x_df['arrival_time'] = date + date_x_df['arrival_time']
        date_x_df['trip_id'] = [f"{trip_id}_{np.datetime_as_string(datetime, unit='D')}" for trip_id, datetime in \
                                zip(date_x_df['trip_id'].values, date_x_df['arrival_time'].values)]

        stop_datetimes_dfs.append(date_x_df)

stop_datetimes = pd.concat(stop_datetimes_dfs)
stop_datetimes = stop_datetimes.sort_values(by=['arrival_time'])
stop_datetimes = stop_datetimes.drop(columns=['service_id'])
stop_datetimes = stop_datetimes.reset_index(drop=True)

print(len(stop_datetimes))
stop_datetimes.head(3)

4809781


,line_name,trip_id,shape_id,stop,arrival_time,previous_stop,next_stop
0,BLUE,3121350_18154_2019-09-14,46,4697,2019-09-14 06:54:00,<NA>,4664
1,BLUE,3121350_18154_2019-09-14,46,4664,2019-09-14 06:57:00,4697,13107
2,GREEN,3120258_18154_2019-09-14,117,21110,2019-09-14 07:00:00,<NA>,10142


In [14]:
stop_datetimes.to_csv(f"Intermediates\\{GTFS_FOLDER}\\gtfs_stop_datetimes.csv", index=False)

# Prepare connections

2. Cut into segements between stations (also segments per direction)

In [15]:
def cut(line: LineString, point: Point):
    # Cuts a line in two at a distance closest to a given point
    distance = line.line_locate_point(point)

    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]

In [16]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
0,3037507_18184,05:00:00,05:00:00,9360,1,0,0,0.0000
1,3037507_18184,05:04:00,05:04:00,7993,2,0,0,2.5847
2,3037507_18184,05:07:00,05:07:00,3969,3,0,0,4.7091
3,3037507_18184,05:10:00,05:10:00,22094,4,0,0,5.8498
4,3037507_18184,05:13:00,05:13:00,3542,5,0,0,7.1824
...,...,...,...,...,...,...,...,...
409085,3102103_18171,05:47:00,05:47:00,4613,19,0,0,13.1411
409086,3102103_18171,05:50:00,05:50:00,1305,20,0,0,14.4528
409087,3102103_18171,05:52:00,05:52:00,13107,21,0,0,15.5504
409088,3102103_18171,05:55:00,05:55:00,4664,22,0,0,16.9608


In [17]:
# Find all individual track segments by splitting shapes by each station along the route
links = gpd.GeoDataFrame(columns=['link_id', 'from_stop', 'to_stop', 'geometry'], geometry='geometry')

warnings.filterwarnings("ignore", message="invalid value encountered in line_locate_point")
trips_per_service = {service_id[0]: service_trip_data for service_id, service_trip_data in trips.groupby(['service_id'])}
stops_per_trip = {trip_id[0]: trip_stop_data for trip_id, trip_stop_data in stop_times.groupby(['trip_id'])}

for service_id, service_trip_data in trips_per_service.items():
    routes = service_trip_data['shape_id'].unique()

    trips_per_route = {route_id[0]: route_trip_data for route_id, route_trip_data in service_trip_data.groupby(['shape_id'])}
    for route_id, route_trip_data in trips_per_route.items():
        single_trip = stops_per_trip[route_trip_data['trip_id'].values[0]]
        route_shape = route_shapes[route_shapes['shape_id'] == route_id].iloc[0]['geometry']

        previous_id = None
        previous_geom = None
        for trip_stop in single_trip['stop_id']:
            if previous_id is None:
                previous_id = trip_stop
                previous_stop_index = stations[stations['stop_id'] == previous_id].index[0]
                previous_geom = stations.at[previous_stop_index, 'geometry']
                continue

            current_id = trip_stop
            current_stop_index = stations[stations['stop_id'] == current_id].index[0]
            current_geom = stations.at[current_stop_index, 'geometry']

            if f"{previous_id}_to_{current_id}" not in links['link_id'].values:

                beginning_cut = cut(route_shape, previous_geom)
                beginning_cut = beginning_cut[-1]

                route_segment = cut(beginning_cut, current_geom)
                route_segment = route_segment[0]
            
                links.loc[len(links)] = [f"{previous_id}_to_{current_id}", previous_id, current_id, route_segment]
            
            previous_id = current_id
            previous_geom = current_geom

print(len(links))
links.head(3)

189


,link_id,from_stop,to_stop,geometry
0,9360_to_7993,9360,7993,"LINESTRING (-77.16476 39.11999, -77.14669 39.0..."
1,7993_to_3969,7993,3969,"LINESTRING (-77.14669 39.08545, -77.12079 39.0..."
2,3969_to_22094,3969,22094,"LINESTRING (-77.12079 39.06239, -77.11278 39.0..."


In [18]:
# Determine travel times between stations via links
stop_times_prepped_2 = stop_times.copy()
stop_times_prepped_2['arrival_time'] = [timedeltas[arrival_time] for arrival_time in stop_times_prepped_2['arrival_time'].values]

travel_times = {link: [] for link in links['link_id'].values}
stop_times_per_trip = {trip_id[0]: trip_stop_data for trip_id, trip_stop_data in stop_times_prepped_2.groupby(['trip_id'])}

for trip_id, trip_stop_times in stop_times_per_trip.items():
    trip_stop_times = trip_stop_times.reset_index(drop=True)

    for i in range(len(trip_stop_times)-1):
        link = f"{trip_stop_times.at[i, 'stop_id']}_to_{trip_stop_times.at[i+1, 'stop_id']}"
        travel_time = (trip_stop_times.at[i+1, 'arrival_time']- trip_stop_times.at[i, 'arrival_time']).seconds

        travel_times[link].append(travel_time)

links['travel_time'] = [None] * len(links)
for link, travel_time_list in travel_times.items():
    link_index = links[links['link_id'] == link].index.to_list()[0]
    links.at[link_index, 'travel_time'] = int(median(travel_time_list))

links.head(5)

,link_id,from_stop,to_stop,geometry,travel_time
0,9360_to_7993,9360,7993,"LINESTRING (-77.16476 39.11999, -77.14669 39.0...",240
1,7993_to_3969,7993,3969,"LINESTRING (-77.14669 39.08545, -77.12079 39.0...",180
2,3969_to_22094,3969,22094,"LINESTRING (-77.12079 39.06239, -77.11278 39.0...",180
3,22094_to_3542,22094,3542,"LINESTRING (-77.11278 39.04716, -77.10384 39.0...",180
4,3542_to_2640,3542,2640,"LINESTRING (-77.10384 39.02924, -77.09695 39.0...",180


In [19]:
links = links.set_crs('EPSG:4326')
links.to_file(f"Intermediates\\{GTFS_FOLDER}\\gtfs_links.geojson", driver="GeoJSON")

# Test plot of stations and links

In [20]:
stops_chart = alt.Chart(stations[['stop_id', 'stop_name', 'geometry']]).mark_geoshape(
).encode(
    tooltip=['stop_id:N', 'stop_name:N'],
).properties(
    width=800,
    height=500
).project('mercator')


links_chart = alt.Chart(links).mark_geoshape(
    filled=False
).encode(
    tooltip=['link_id'],
    stroke=alt.Color('link_id:N', legend=None),
    strokeWidth= alt.value(3)
).properties(
    width=800,
    height=500
).project('mercator')

links_chart + stops_chart

alt.LayerChart(...)